In [1]:
import odc.stac
import pystac
from odc.geo.geobox import AnchorEnum

In [2]:
item = pystac.Item.from_file("MOD21A1D.A2019190.h10v02.061.2020303070212/MOD21A1D.A2019190.h10v02.061.2020303070212.json")

Success when we load and reproject in one call:

In [3]:
ds = odc.stac.load(
    [item],
    chunks={"x": 2048, "y": 2048},
    crs="EPSG:32603",
    anchor=AnchorEnum.CENTER,
)
ds

<xarray.Dataset> Size: 41MB
Dimensions:        (y: 2703, x: 3762, time: 1)
Coordinates:
  * y              (y) float64 22kB 9.165e+06 9.164e+06 ... 6.662e+06 6.662e+06
  * x              (x) float64 30kB -1.614e+06 -1.613e+06 ... 1.87e+06 1.871e+06
    spatial_ref    int32 4B 32603
  * time           (time) datetime64[ns] 8B 2019-07-09
Data variables:
    lst_view_time  (time, y, x) float32 41MB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>

However, we moved to a load and then reproject order of operations to work around a
[small data gap when loading adjacent sinusoidal
tiles](https://github.com/opendatacube/odc-stac/issues/150) that do not overlap. When we
use the load and then reproject order, we get a failure. Suspicious that it is related
to the antimeridian.

In [4]:
ds = odc.stac.load(
    [item],
    chunks={"x": 2048, "y": 2048},
).odc.reproject("EPSG:32603", anchor=AnchorEnum.CENTER)
ds

GEOSException: TopologyException: side location conflict at 156.37765057717081 59.191953625054566. This can occur if the input geometry is invalid.

Note that if do not pass in chunks, we _can_ load and then reproject successfully.
However, we are using Dask and need to specify chunk size.

In [5]:
ds = odc.stac.load(
    [item],
).odc.reproject("EPSG:32603", anchor=AnchorEnum.CENTER)
ds

<xarray.Dataset> Size: 41MB
Dimensions:        (time: 1, y: 2707, x: 3766)
Coordinates:
  * time           (time) datetime64[ns] 8B 2019-07-09
  * y              (y) float64 22kB 9.169e+06 9.168e+06 ... 6.662e+06 6.662e+06
  * x              (x) float64 30kB -1.615e+06 -1.614e+06 ... 1.874e+06
    spatial_ref    int32 4B 32603
Data variables:
    lst_view_time  (time, y, x) float32 41MB nan nan nan nan ... nan nan nan nan